In [1]:
%matplotlib inline

In [98]:
import xlrd
import os 
import sqlite3
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

sns.set(style="whitegrid")

In [99]:
db_path = '../datasets/datawarehouse.db'

In [109]:
con = sqlite3.connect(db_path)
cur = con.cursor()

query = '''
    SELECT NBHDNAME, AVG(ASSESSMENT),
      CASE
        WHEN AVG(ASSESSMENT) >= 1000000 THEN 'high' 
        WHEN AVG(ASSESSMENT) >= 600000 THEN 'medium' 
        ELSE 'low' 
      END AS assessment_tier
    FROM property_info
    WHERE PROPTYPE LIKE '%Residential%'
    GROUP BY NBHDNAME
    ORDER BY AVG(ASSESSMENT) DESC;'''

cur.execute(query)

avg_assess = cur.fetchall()
con.close()

In [110]:
assess_df = pd.DataFrame(avg_assess, columns=['neighborhood', 'price', 'label'])
nbhd_group = []
avg_assessment = []
label = []
for p in avg_assess:
    nbhd_dict[p[0]] = p[2]
    
nbhd_dict['Shaw, Logan Circle'] = 'medium'
nbhd_dict['Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street'] = 'medium'
nbhd_dict['River Terrace, Benning, Greenway, Dupont Park'] = 'medium'
nbhd_dict['Eastland Gardens, Kenilworth'] = 'low'
nbhd_dict['Douglas, Shipley Terrace'] = 'low'

In [122]:
con = sqlite3.connect(db_path)
cur = con.cursor()

query = '''
    SELECT neighbourhood_cleansed, CAST(REPLACE(price, '$', '') AS FLOAT), bathrooms_text
    FROM listings;'''

cur.execute(query)

listings = cur.fetchall()
con.close()

In [125]:
listings_df = pd.DataFrame(listings, columns = ['neighborhood', 'price', 'bathrooms'])
# Create a new column "neighborhood_label" based on the values in the "neighborhood" column
listings_df['neighborhood_label'] = listings_df['neighborhood'].map(nbhd_dict)

In [131]:
# create mapping for shared/private and number of baths
bath_mapping = {
    'private': {
        '1 bath': 1,
        '1.5 baths': 1.5,
        '1 private bath': 1,
        '2 baths': 2,
        '2.5 baths': 2.5,
        '3 baths': 3,
        '3.5 baths': 3.5,
        '4 baths': 4,
        '4.5 baths': 4.5,
        '5 baths': 5,
        '5.5 baths': 5.5,
        '6 baths': 6,
        '7 baths': 7,
        '7.5 baths': 7.5,
        '11 shared baths': 11,
        '15 baths': 15,
        '0 baths': 0
    },
    'shared': {
        '0 shared baths': 0,
        '1 shared bath': 1,
        '1.5 shared baths': 1.5,
        '2 shared baths': 2,
        '2.5 shared baths': 2.5,
        '3 shared baths': 3,
        '3.5 shared baths': 3.5,
        '4 shared baths': 4,
        '4.5 shared baths': 4.5,
        '5 shared baths': 5,
        '5.5 shared baths': 5.5,
        '6 shared baths': 6,
        '8 shared baths': 8,
        '11 shared baths': 11,
        'Half-bath': 0.5,
        'Shared half-bath': 0.5
    }
}

# create new column for shared/private
listings_df['shared_private'] = listings_df['bathrooms'].apply(lambda x: 'private' if 'private' in x.lower() else 'shared')

# create new column for number of baths
listings_df['num_baths'] = listings_df['bathrooms'].apply(lambda x: bath_mapping[listings_df['shared_private'][0]].get(x, None))

listings_df['shared_private'].fillna(listings_df['shared_private'].mode()[0], inplace=True)
listings_df['num_baths'].fillna(listings_df['num_baths'].median(), inplace=True)
listings_df = listings_df.drop('neighborhood', axis=1)

In [132]:
listings_df

,price,bathrooms,neighborhood_label,shared_private,num_baths
0,20.0,1 private bath,medium,private,1.0
1,185.0,2.5 baths,low,shared,2.5
2,221.0,2 baths,medium,shared,2.0
3,142.0,1 bath,medium,shared,1.0
4,398.0,2 baths,high,shared,2.0
...,...,...,...,...,...
8623,70.0,1 bath,medium,shared,1.0
8624,253.0,1 bath,high,shared,1.0
8625,95.0,1 bath,high,shared,1.0
8626,180.0,1 bath,high,shared,1.0


In [ ]:
listings_df.to_csv('../datasets/')